**Welcome! This is my first public notebook ever, and i want to share with you. Feel free to comment anything that should be better. I'm beggining in Data Science!**

In [ ]:
# ALCIDES GABRIEL - 2021
# THE TITANIC TASK FROM KAGGLE

### Importing library

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from yellowbrick.classifier import ConfusionMatrix
import statistics as sts
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import matplotlib.pyplot as plt

### Loading and cleaning the data

In [ ]:
base = pd.read_csv('../input/titanic/train.csv')
base.shape

In [ ]:
base

In [ ]:
base.isnull().sum()

In [ ]:
# Replacing the NaN values in Age column with the median
med = np.nanmedian(base['Age'])
base['Age'].fillna(med, inplace=True)
base.isnull().sum()

In [ ]:
# Replacing the NaN values in Embarked column with the mode
base.groupby(['Embarked']).count()
base['Embarked'].fillna('S', inplace=True)
base.isnull().sum()

In [ ]:
del base['Cabin']

### Graphical Exploratory Analysis

In [ ]:
base.groupby('Sex').count().reset_index()
plt.bar(base.groupby('Sex').count().reset_index()['Sex'], base.groupby('Sex').count().reset_index()['PassengerId'], color='gray', edgecolor='k')
plt.title('Titanic passengers by sex')
plt.grid(axis='x')
plt.show()

In [ ]:
pclas = base.groupby('Pclass').count().reset_index()
pclas.loc[pclas['Pclass'] == 1, 'Pclass'] = 'First'
pclas.loc[pclas['Pclass'] == 2, 'Pclass'] = 'Second'
pclas.loc[pclas['Pclass'] == 3, 'Pclass'] = 'Third'
plt.bar(pclas['Pclass'], pclas['PassengerId'], color='gray', edgecolor='k')
plt.title('Titanic passengers by class')
plt.grid(axis='x')
plt.show()

In [ ]:
plt.hist(base['Age'], bins=10, color='gray', edgecolor='k')
plt.xlabel('Age group')
plt.grid(axis='x')
plt.title('Age group of Titanic passengers')
plt.show()

In [ ]:
base.loc[base['Embarked'] == 'C', 'Embarked'] = 'Cherbourg'
base.loc[base['Embarked'] == 'Q', 'Embarked'] = 'Queenstown'
base.loc[base['Embarked'] == 'S', 'Embarked'] = 'Southampton'
plt.bar(base.groupby(['Embarked']).count().reset_index()['Embarked'], base.groupby(['Embarked']).count().reset_index()['PassengerId'], color='gray', edgecolor='k')
plt.title('Port of Embarkation')
plt.grid(axis='x')

In [ ]:
surv = ['Didnt Survived', 'Survived']
plt.bar(surv, base.groupby('Survived').count().reset_index()['PassengerId'], color='gray', edgecolor='k')
plt.grid(axis='x')
plt.title('Titanic passengers')
plt.show()

In [ ]:
sns.boxplot(base['Age'], color='gray')
plt.xlabel('Titanic passengers age')
plt.show()

In [ ]:
sns.boxplot(base['Fare'], color='gray')
plt.xlabel('Passengers fare')
plt.show()

In [ ]:
base.describe()

### Pre-processing

In [ ]:
# Normalizing some data
base['Age'] = (base['Age']-min(base['Age']))/(max(base['Age'])-min(base['Age']))
base['Fare'] = (base['Fare']-min(base['Fare']))/(max(base['Fare'])-min(base['Fare']))

In [ ]:
del base['Ticket']
del base['Name']

In [ ]:
predictors = base.iloc[:,2:].values
tclass = base.iloc[:,1].values
predictors

In [ ]:
labelencoder1 = LabelEncoder()
predictors[:,1] = labelencoder1.fit_transform(predictors[:,1])
labelencoder2 = LabelEncoder()
predictors[:,6] = labelencoder2.fit_transform(predictors[:,6])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(predictors,
                                                       tclass,
                                                       test_size=0.3,
                                                       random_state=0)

### Training the models

In [ ]:
# Naive Bayes
naive = GaussianNB()
naive.fit(X_train, y_train)
NBprev = naive.predict(X_test)
NBscore = accuracy_score(y_test, NBprev)
NBconf = confusion_matrix(y_test, NBprev)
nbtitle = 'Confusion Matrix: Naive Bayes, score: ', NBscore
sns.heatmap(NBconf, annot=True, fmt='g', cmap='Blues')
plt.title(nbtitle)
plt.show()

In [ ]:
# Decision Tree
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
DTprev = tree.predict(X_test)
DTscore = accuracy_score(y_test, DTprev)
DTconf = confusion_matrix(y_test, DTprev)
dttitle = 'Confusion Matrix: Decision Tree, score: ', DTscore
sns.heatmap(DTconf, annot=True, fmt='g', cmap='Blues')
plt.title(dttitle)
plt.show()

In [ ]:
# Random Forest
forest = RandomForestClassifier(n_estimators=1000)
forest.fit(X_train, y_train)
RFprev = forest.predict(X_test)
RFscore = accuracy_score(y_test, RFprev)
RFconf = confusion_matrix(y_test, RFprev)
rftitle = 'Confusion Matrix: Random Forest, score: ', RFscore
sns.heatmap(RFconf, annot=True, fmt='g', cmap='Blues')
plt.title(rftitle)
plt.show()

In [ ]:
# Logistic Regression
reglog = LogisticRegression()
reglog.fit(X_train, y_train)
RLprev = reglog.predict(X_test)
RLscore = accuracy_score(y_test, RLprev)
RLconf = confusion_matrix(y_test, RLprev)
rltitle = 'Confusion Matrix, score: ', RLscore
sns.heatmap(RLconf, annot=True, fmt='g', cmap='Blues')
plt.title(rltitle)
plt.show()

### Re-building dataframe with RandomForest results

In [ ]:
conc = pd.DataFrame(X_test, columns=['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])
conc['Survived'] = RFprev
conc

In [ ]:
predsex = labelencoder1.inverse_transform([0,1])
print(predsex)

In [ ]:
conc['Sex'] = conc.apply(lambda x: 'female' if x['Sex'] == 0 else 'male', axis=1)

In [ ]:
predemb = labelencoder2.inverse_transform([0,1,2])
predemb

In [ ]:
conc.loc[conc['Embarked'] == 0, 'Embarked'] = 'Cherbourg'
conc.loc[conc['Embarked'] == 1, 'Embarked'] = 'Queenstown'
conc.loc[conc['Embarked'] == 2, 'Embarked'] = 'Southampton'

In [ ]:
conc['Age'] = conc['Age'] * (80 - 0.42) + 0.42
conc['Fare'] = conc['Fare'] * (512.3292 - 0)
conc

### Machine learning conclusions

In [ ]:
srv = ['Didnt survived', 'Survived']
plt.bar(srv, conc.groupby(['Survived']).count().reset_index()['Age'], color='gray', edgecolor='k')
plt.grid(axis='x')
plt.title('Titanic passengers')
plt.show()

In [ ]:
surv = conc.loc[conc['Survived'] == 1]
plt.bar(surv.groupby(['Sex']).count().reset_index()['Sex'], surv.groupby(['Sex']).count().reset_index()['Pclass'], color='gray', edgecolor='k')
plt.title('Titanic survivors by sex')
plt.grid(axis='x')
plt.show()

In [ ]:
plt.hist(surv['Age'], color='gray', edgecolor='k')
plt.grid(axis='x')
plt.title('Age group of Titanic survivors')
plt.xlabel('Age group')
plt.show()

In [ ]:
pcsurv = ['First', 'Second', 'Third']
plt.bar(pcsurv, surv.groupby('Pclass').count().reset_index()['Sex'], color='gray', edgecolor='k')
plt.title('Titanic survivors by class')
plt.grid(axis='x')
plt.show()

In [ ]:
plt.bar(surv.groupby('Embarked').count().reset_index()['Embarked'], surv.groupby('Embarked').count().reset_index()['Sex'], color='gray', edgecolor='k')
plt.title('Titanic survivors by port of embarkation')
plt.grid(axis='x')
plt.show()

In [ ]:
sns.boxplot(surv['Age'], color='gray')
plt.title('Titanic survivors age')
plt.xlabel('Age')
plt.show()

In [ ]:
surmea = sts.mean(surv['Age'])
surmed = sts.median(surv['Age'])
print('Mean of Titanic survivors age: ', surmea, '| Median of Titanic survivors age: ', surmed)

### Export results to Kaggle

In [ ]:
test = pd.read_csv('../input/titanic/test.csv')
test.isnull().sum()

In [ ]:
medt = np.nanmedian(test['Age'])
medtf = np.nanmedian(test['Fare'])
test['Age'].fillna(medt, inplace=True)
test['Fare'].fillna(medt, inplace=True)
test.isnull().sum()

In [ ]:
del test['Name']
del test['Ticket']
del test['Cabin']

In [ ]:
tmax = max(test['Age'])
tmin = min(test['Age'])
fmax = max(test['Fare'])
fmin = min(test['Fare'])
test['Age'] = (test['Age']-tmin)/(tmax-tmin)
test['Fare'] = (test['Fare']-fmin)/(fmax-fmin)
test

In [ ]:
test.loc[test['Embarked'] == 'C', 'Embarked'] = 'Cherbourg'
test.loc[test['Embarked'] == 'Q', 'Embarked'] = 'Queenstown'
test.loc[test['Embarked'] == 'S', 'Embarked'] = 'Southampton'

In [ ]:
prevtest = test.iloc[:,1:].values
prevtest[:,1] = labelencoder1.transform(prevtest[:,1])
prevtest[:,6] = labelencoder2.transform(prevtest[:,6])
prevkag = forest.predict(prevtest)

In [ ]:
kaggle = pd.DataFrame()
kaggle['PassengerId'] = test['PassengerId']
kaggle['Survived'] = prevkag
kaggle

In [ ]:
kaggle.to_csv('kaggle.csv', index=False)